In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows() # Disable 5_000 rows limit
findings_data = pd.read_csv("https://code4rena.com/community-resources/findings.csv") # Set path accordingly
findings_data["contestid"] = findings_data["contest"]
contests_data = pd.read_csv("https://code4rena.com/community-resources/contests.csv")

In [2]:
df = pd.merge(findings_data, contests_data[["contestid", "end_time"]], on="contestid").drop_duplicates()
df["end_time"] = pd.to_datetime(df["end_time"].str[:25])
df["risk_label"] = df["risk"].map(
    {
        '0': '0_Very low (unused since February 2022)', 
        '1': '1_Low (unused since February 2022)',
        '2': '2_Medium',
        '3': '3_High',
        'g': 'g_Gas optimization',
        'q': 'q_QA report',
    }
)

In [3]:
ordered_legend_reports_labels = [
    '3_High',
    '2_Medium',
    'g_Gas optimization',
    'q_QA report',
    '1_Low (unused since February 2022)',
    '0_Very low (unused since February 2022)', 
]
label_colors = ["#FE266D","#FA6C44","#F2E713","#D1D811","#0AB6F8","#5688C1"]
chart_width = 850
chart_height = 350

In [4]:
alt.Chart(df.groupby(["risk_label", "split"])["awardUSD"].median().reset_index(),
          title="Findings' value distribution according to number of shared submissions by risk level (for rewards > 50 $USD)"
).transform_filter(
    (alt.datum.risk_label != "0_Very low (unused since February 2022)") & (alt.datum.awardUSD > 50)
).mark_bar().encode(
    x=alt.X("split:O", title="Number of wardens sharing a finding", axis=alt.Axis()),
    y=alt.Y("awardUSD:Q", title="Finding $USD value", axis=alt.Axis(format='$,.0f')),
    color=alt.Color(
        'risk_label:N', 
        title="Risk level",
        scale=alt.Scale(domain=ordered_legend_reports_labels[:-1], range=label_colors[:-1]),
        legend=alt.Legend(orient="top", labelFontSize=12, labelLimit=250),
    ),
    row=alt.Row("risk_label:N", sort=ordered_legend_reports_labels[:-1], title=""),
    tooltip=["risk_label:N", "split:O", "awardUSD:Q"]
).resolve_scale(
    y='independent'
).resolve_axis(
    x='independent'
).interactive()

alt.Chart(...)

In [5]:
rewards = alt.Chart(
    df.groupby([df.end_time.dt.to_period("M"), "risk_label"])["awardUSD"].mean().reset_index().astype({"end_time": str}), 
    width=chart_width, 
    height=chart_height,
    title="Value of a submission ($USD) over time by risk level"
).mark_line(
    point=True
).encode(
    x=alt.X('yearmonth(end_time):T', title=""),
    y=alt.Y('awardUSD:Q', title="", axis=alt.Axis(format='$,.0f')),
    color=alt.Color(
        'risk_label:N', 
        title="Risk level",
        scale=alt.Scale(domain=ordered_legend_reports_labels, range=label_colors),
        legend=alt.Legend(orient="top", labelFontSize=12, labelLimit=250)
    ),
    tooltip=['end_time:T', 'risk_label:N', 'awardUSD:Q']
)

In [6]:
submissions = alt.Chart(
    df.groupby([df.end_time.dt.to_period("M"), "risk_label"])["contest"].count().reset_index().astype({"end_time": str}),
    width=chart_width,
    height=chart_height,
    title="Expected warden reward ($USD) and number of reports over time by risk level",
).mark_line(
    opacity=.75,
    strokeDash=[2]
).encode(
    x=alt.X('yearmonth(end_time):T', title=""),
    y=alt.Y('contest:Q', title="Number of reports"),
    color=alt.Color(
        'risk_label:N', 
        title="Risk level",
        scale=alt.Scale(domain=ordered_legend_reports_labels, range=label_colors),
        legend=alt.Legend(orient="top", labelFontSize=12, labelLimit=250),
    ),
)

## Expected warden reward calculation

1. For each contest, take the total number of submissions for each risk level and divide it by the number of participants. This gives a value *X* that corresponds to **the average number of submission for each risk level by a single warden**.
2. Now take the mean reward value for each contest and risk level and multiply that by *X*. This gives a value *Y* that corresponds to **the expected warden reward according the average number of submission**.
3. Group the contests by their ending date (month/year) and take the mean of the *Y*s. This gives **the expected warden reward per month for each risk level** which is what is plotted in the second chart. 

In [7]:
df2 = pd.merge(df.groupby(["contest", "end_time", "risk"])[["finding"]].count().reset_index(), df.groupby("contest")["handle"].nunique(), on="contest")

In [8]:
df2["average_findings"] = df2.finding / df2.handle
df2["end_time"] = pd.to_datetime(df2["end_time"])

In [9]:
df3 = pd.merge(df, df2[["contest", "average_findings", "risk"]], on=["contest", "risk"])

In [10]:
df3["average_usd_per_risk"] = df3.average_findings * df3.awardUSD
df3[["end_time", "contest", "average_findings", "risk", "risk_label", "awardUSD", "average_usd_per_risk"]]

,end_time,contest,average_findings,risk,risk_label,awardUSD,average_usd_per_risk
0,2021-02-22 23:59:00,1,1.000000,g,g_Gas optimization,191.49,191.490
1,2021-02-22 23:59:00,1,1.000000,g,g_Gas optimization,191.49,191.490
2,2021-02-22 23:59:00,1,1.000000,g,g_Gas optimization,191.49,191.490
3,2021-02-22 23:59:00,1,1.000000,g,g_Gas optimization,191.49,191.490
4,2021-02-22 23:59:00,1,1.000000,g,g_Gas optimization,191.49,191.490
...,...,...,...,...,...,...,...
37840,2023-11-17 20:00:00,306,0.366667,q,q_QA report,4.08,1.496
37841,2023-11-17 20:00:00,306,0.366667,q,q_QA report,4.08,1.496
37842,2023-11-17 20:00:00,306,0.366667,q,q_QA report,4.08,1.496
37843,2023-11-17 20:00:00,306,0.366667,q,q_QA report,4.08,1.496


In [11]:
weighted_rewards = alt.Chart(df3.groupby([df3.end_time.dt.to_period("M"), "risk_label"])["average_usd_per_risk"].mean().reset_index().astype({"end_time": str}),
          width=chart_width,
          height=chart_height
).mark_line(point=True).encode(
    x='yearmonth(end_time):T',
    y=alt.Y('average_usd_per_risk:Q', title="", axis=alt.Axis(format='$,.0f')),
    color=alt.Color(
        'risk_label:N', 
        title="Risk level",
        scale=alt.Scale(domain=ordered_legend_reports_labels, range=label_colors),
        legend=alt.Legend(orient="top", labelFontSize=12, labelLimit=250)
    ),
    tooltip=['end_time:T', 'risk_label:N', 'average_usd_per_risk:Q']
)

## Analysis

While the hierarchy for a submission value is well respected over time (as shown in the first graph), the expected value tells another story about which category is more worth in the eyes of wardens. With the amount of reports increasing, the expected reward for each category tends to uniformize and it's not clear which of the *mediums* or *highs* findings are more worth than the other.

Certainly, the though competiton of recent contests has made the value of *high* and *medium* findings diminish which is not particularly a good sign since their edging closer to the values of easier and potentially automated *gas optimization* and *QA reports*.

In [12]:
(rewards & (weighted_rewards + submissions).resolve_scale(y='independent')).interactive()

alt.VConcatChart(...)